Seminar Stochastik in Aktion, Johannes Schade

# Walker's Trick (Alias Method)

First, we import the necessary libraries and methods:

In [ ]:
import numpy as np
from math import exp, factorial
import matplotlib.pyplot as plt

Define the Poisson probability function 
$p_\lambda(k) = \lambda^k \frac{e^{-\lambda}}{k!}$:

In [ ]:
def poisson (k, lam=1):
    return lam**k * exp(-lam)/factorial(k)

In [ ]:
# Set cut-off value for infinite discrete probability distribution:
n=20

# Store first n-2 values
d = [poisson(i, lam = 4) for i in range(n-1)]
# append last value s.t. sum(p)=1 to achieve proper probablity distribution
d.append(1-sum(d))

print(f'd={d}')

Plot for plausibility:

In [ ]:
plt.bar([i for i in range(len(d))], d)

Create alias table (in form of the two rows `probs` and `alias`')

In [ ]:
def alias_table(d):
    """
    :param list d: list of probabilities of probability distribution
    :return probs, alias: alias table for Walker's alias method
    """
    # convert to numpy.array for convenient methods
    d = np.array(d)
    n_d = n * d

    # initialize probs-alias-table
    probs = np.array(n * [None])
    alias = np.zeros(n, dtype='int32')

    for i in range(n_d.size - 1):
        # choose index (col) with area left
        lesser = np.where(n_d < 1)[0][0]  
        # choose index (col) with area overshoot
        greater = np.where(n_d >= 1)[0][0]
        # save save Pr(l) to probs[l]
        probs[lesser] = n_d[lesser]
        # save index of overshoot probability to alias[l]
        alias[lesser] = greater
        # reduce n_d[g] by probability mass distributed to alias[l]
        n_d[greater] = n_d[greater] - (1 - n_d[lesser])
        # remove probability mass used in probs[l]
        n_d[lesser] = None

    # take care of last probability left over
    probs[np.where(probs == None)[0][0]] = 1
    
    return probs, alias

With `probs` and `alias` we can sample by using only the $\mathcal{U}_{[0,1)}$ generator `numpy.random.uniform(0,1)`:


In [ ]:
def alias_sample(probs, alias):
    """
    :param probs: probability row of alias table
    :param alias: alias row of alias table
    :return:
    """
    omega = n*np.random.uniform(0,1) # cont. Uniform distribution
    rrow = int(omega) # discrete Uniform distribution
    rcol = omega - rrow

    if rcol < probs[rrow]: return rrow
    else: return alias[rrow]

By composing the functions `alias_table` and `alias_sample` we obtain an implementation of the alias method:

In [ ]:
def alias_method(d):
    """
    :param p: list of probabilities of probability distribution
    :return: sample from probability distribution
    """
    probs, alias = alias_table(d)
    return alias_sample(probs, alias)

Checking sample mean and sample variance for plausibility:

In [ ]:
# initialize sample array:
sample = []
for i in range(10**4):
    sample.append(alias_method(d))

# calculate mean and variance
print(f'mean = {np.mean(sample)}, variance = {np.var(sample)}')

# Application to the epidemic

We assume probabilites $r_i$ of transmitting a disease on te $i$-th day of infection:

In [ ]:
r = np.array([0, 0, 0, 0, 0.1, 0.3, 0.4, 0.4, 0.2, 0])

For every day $i$, we sample several $p(i)$ with $\lambda = 4$ for the number of contacts:

In [ ]:
n_samples=10**3
c = np.zeros((n_samples, 10))
for i in range (n_samples):
    for j in range(10):
        c[i,j] = alias_method(d)

Calculate $\sum_{i=0}^{9} r_iX_i = \mathbf{r}^\intercal \mathbf{X}$ for every of the "indivdual samples":

In [ ]:
for i in range(10):
    print(f'Average number of people infected by individual {i}: {np.dot(r, c[i])}')

Average over all samples:

In [ ]:
print(np.mean(np.matmul(c,r)))

Compare to
$$ \mathrm{E}[\mathbf{r}^\intercal \mathbf{X}] = \mathrm{E} \left[\sum_{i=0}^{9} r_iX_i \right] = \mathrm{r}^\intercal \mathrm{E}\mathbf{X} = \mathrm{r}^\intercal \mathbf{1} \lambda = \mathrm{r}^\intercal \mathbf{4} \approx 5.6$$ 